In [2]:
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

```Python
def seriesIVERCAM(file):
    """Importa los archivos csv con la serie temporal de mapas climáticos generados en IVERCAM.
    
    Entradas:
    ---------
    file:    string. Nombre del archivo
    
    Salidas:
    --------
    arr:    array (dates,X,Y). Matriz con los datos
    X:      array. Coordenadas X del mapa
    Y:      array. Coordenadas Y del mapa
    dates:  array. Fechas de cada mapa
    """
    
    # importar csv
    data = pd.read_csv(file)

    # coordenadas X e Y
    XX = data.X
    X = np.sort(np.unique(XX))
    YY = data.Y
    Y = np.sort(np.unique(YY))[::-1]
    nrow, ncol = len(Y), len(X)
    
    # fechas
    data_ = data.drop(['X', 'Y'], axis=1)
    dates = list(data_.columns)
    ntime = len(dates)
    
    # array 3D
    arr = np.empty((ntime, nrow, ncol))
    arr[:,:,:] = np.nan
    for n in range(data_.shape[0]):
        i = np.where(Y == YY[n])[0][0]
        j = np.where(X == XX[n])[0][0]
        arr[:,i,j] = data_.iloc[n,:]

    arr = ma.masked_where(np.isnan(arr), arr)
    
    return arr, X, Y, dates

# precipitación
pcp, Xp, Yp, Tp = seriesIVERCAM('pcp_pred_1000.csv')

plt.figure()
im = plt.imshow(pcp.mean(axis=0))
cb = plt.colorbar(im, shrink=.75)
plt.axis('off');

# temperatura
tmp, Xt, Yt, Tt = seriesIVERCAM('Tmed_pred_1000_IDWz.csv')

plt.figure()
im = plt.imshow(tmp.mean(axis=0))
cb = plt.colorbar(im, shrink=.75)
plt.axis('off');
```

In [17]:
def interpolarNN(XXorig, YYorig, data, XXgrid, YYgrid, n_neighbors=1, weights='distance', p=1):
    """Interpola una malla regular a partir de unos datos de otra malla mediante el método de la distancia inversa ponderada
    
    Entrada:
    --------
    XXorig:      array(n,). Coordenada X de todas las celdas de la malla de partida
    YYorig:      array(n,). Coordenada Y de todas las celdas de la malla de partida
    data:        masked array (i,j). Mapa original
    XXgrid:      array(m,). Coordenada X de todas las celdas de la malla a la que se quiere interpolar
    YYgrid:      array(m,). Coordenada Y de todas las celdas de la malla a la que se quiere interpolar
    n_neighbors: int. Nº de celdas cercanas a tener en cuenta en la interpolación
    weights:     str. Método para calcular el peso
    p:           float. Exponente de ponderación del peso
    
    Salida:
    -------
    pred:        masked array (k,l). Mapa interpolado
    """
    
    # AJUSTE
    # ......
    # target array
    if np.ma.isMaskedArray(data):
        y = data.data.flatten().astype(float)
        mask = data.mask.flatten()
    else:
        y = data.flatten()
        mask = np.isnan(y)
    y = y[~mask]
    # feature matrix
    X = np.vstack((XXorig.flatten(), YYorig.flatten())).T
    X = X[~mask,:]
    # definir y ajustar el modelo
    neigh = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=p).fit(X, y)
    
    # PREDICCIÓN
    # ..........
    # feauture matrix
    X_ = np.vstack((XXgrid.flatten(), YYgrid.flatten())).T
    # predecir
    pred = neigh.predict(X_).reshape((len(Ygrid), len(Xgrid)))
    
    return pred

In [1]:
def recortarRaster(data, X, Y, polygon, crs=None, buffer=None):
    """Recorta los datos 3D según un shapefile de polígono.
    
    Entradas:
    ---------
    data:    array (T,Y,X). Serie temporal de mapas
    X:       array (X,). Coordenadas X de las columnas del mapa
    Y:       array (Y,). Coordenadas Y de las filas del mapa
    polygon: geopandas.Series. Polígono con el que recortar los datos
    crs:     Sistema de coordenadas de X e Y
    buffer:  boolean. Distancia a la que hacer el buffer. Por defecto es None y no se hace
    
    Salidas:
    --------
    data_:   array (T,Y_,X_). Serie temporal de mapas recortados
    X_:      array. Coordenadas X de las columnas del mapa 'data_'
    Y_:      array. Coordenadas Y de las filas del mapa 'data_' 
    """
    
    # crear buffer de la cuenca
    if buffer is not None:
        mask_shp = polygon.buffer(buffer)
    else:
        mask_shp = polygon
        
    # GeoDataFrame de puntos de la malla HARMONIE reproyectada
    XX, YY = np.meshgrid(X, Y)
    if crs is None:
        crs = cuenca.crs
    points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(XX.flatten(), YY.flatten(),
                                                          crs=crs))

    # máscara con el área de la CHC
    inp, res = mask_shp.sindex.query_bulk(points.geometry, predicate='intersects')
    mask1D = np.isin(np.arange(len(points)), inp)
    mask2D = mask1D.reshape(XX.shape)
    
    if len(data.shape) == 2:
        # recortar 'data' a la cuenca
        data_ = data.copy()
        data_[~mask2D] = np.nan
        
        # eliminar filas y columnas sin datos
        maskR = np.isnan(data_).all(axis=1)
        maskC = np.isnan(data_).all(axis=0)
        data_ = data_[~maskR,:][:,~maskC]
        
    elif len(data.shape) == 3:
        # máscara 3D a partir de la anterior
        mask3D = np.zeros(data.shape, dtype=bool)
        mask3D[:,:,:] = mask2D[np.newaxis,:,:]

        # recortar 'data' a la cuenca
        data_ = data.copy()
        data_[~mask3D] = np.nan

        # eliminar filas y columnas sin datos
        maskR = np.isnan(data_.sum(axis=0)).all(axis=1)
        maskC = np.isnan(data_.sum(axis=0)).all(axis=0)
        data_ = data_[:,~maskR,:][:,:,~maskC]
    
    data_ = np.ma.masked_invalid(data_)
    Y_ = Ygrid[~maskR]
    X_ = Xgrid[~maskC]
    
    return data_, X_, Y_

```Python
def recortar3D(data, X, Y, mask, buffer=None):
    """Recorta los datos 3D según una máscara 2D.
    
    Entradas:
    ---------
    data:    array (T,Y,X). Serie temporal de mapas
    X:       array. Coordenadas X de las columnas del mapa
    Y:       array. Coordenadas Y de las filas del mapa
    buffer:  boolean. Ditancia a la que hacer el buffer. Por defecto es None y no se hace
    
    Salidas:
    --------
    data_:   array (T,Y_,X_). Serie temporal de mapas recortados
    X_:      array. Coordenadas X de las columnas del mapa 'data_'
    Y_:      array. Coordenadas Y de las filas del mapa 'data_' 
    """
    
    # extraer forma de la máscara
    shape = cuenca.shapeRecords()[0]
    
    # coordenadas x e y de cada vértice del polígono
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    xy = [(x_, y_) for x_, y_ in zip(x, y)]

    
    # crear buffer de la cuenca
    if buffer is not None:
        # crear polígono de la cuenca
        poly_cuenca = Polygon(xy)
        # hacer buffer
        buff_cuenca = poly_cuenca.buffer(buffer)
        # extraer coordenadas de los vértices
        X, Y = buff_cuenca.exterior.coords.xy
        XY = [(x, y) for x, y in zip(X, Y)]
        
    # crear 'path' de la cuenca
    path_cuenca = path.Path(xy)
    
    # puntos de cada celda del mapa
    XX, YY = np.meshgrid(X, Y)
    points = np.array((XX.flatten(), YY.flatten())).T

    # máscara 2D con las celdas dentro de la máscara
    mask2D = path_cuenca.contains_points(points).reshape(XX.shape)

    # máscara 3D a partir de la anterior
    mask3D = np.zeros(data.shape, dtype=bool)
    mask3D[:,:,:] = mask2D[np.newaxis,:,:]

    # recortar 'data' a la cuenca
    data_ = data.copy()
    data_[~mask3D] = np.nan

    # eliminar filas y columnas sin datos
    maskR = np.isnan(data_.sum(axis=0)).all(axis=1)
    maskC = np.isnan(data_.sum(axis=0)).all(axis=0)
    data_ = data_[:,~maskR,:][:,:,~maskC]
    data_ = np.ma.masked_invalid(data_)
    Y_ = Y[~maskR]
    X_ = X[~maskC]
    
    return data_, X_, Y_, x, y
```